In [1]:
import pandas as pd
import numpy as np

In [2]:
!apt install libomp-dev
!pip install faiss-gpu # if you don't have a NVIDA GPU use pip install faiss

/bin/bash: apt: command not found
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import faiss

In [4]:
products_train = pd.read_csv("./data/products_train.csv")

In [5]:
sessions_test_task1 = pd.read_csv("./data/sessions_test_task1.csv")

In [6]:
products_de = products_train[products_train["locale"] == "DE"][["id", "title"]]
products_uk = products_train[products_train["locale"] == "UK"][["id", "title"]]
products_jp = products_train[products_train["locale"] == "JP"][["id", "title"]]

In [7]:
products_jp.head()

,id,title
518327,B0BCFTG541,ワタシガモテナイノハドウカンガエテモオマエラガワルイ022 (デジタルバンガンガンコミックス...
518328,B096F11HV9,"Apple MacBook Air M1 2020(13インチAir,8GB RAM,256..."
518329,B0178VYR6Q,キユーピー やさしい献立 なめらかごはん 150g×6個【区分4:かまなくてよい】
518330,B09VWM3SCD,Coticam outdoor ランタンシェード LED ライト ランタン キャンプ アウト...
518331,B07KFZPKLV,Colorsroom こたつ布団 正方形 コタツ布団 185×185cm コタツふとん 掛け...


In [8]:
jp_embeddings = pd.read_csv("/pfs/work7/workspace/scratch/ma_ndoering-data_mining_2/jp_embeddings.csv")
de_embeddings = pd.read_csv("/pfs/work7/workspace/scratch/ma_ndoering-data_mining_2/de_embeddings.csv")
uk_embeddings = pd.read_csv("/pfs/work7/workspace/scratch/ma_ndoering-data_mining_2/uk_embeddings.csv")

In [9]:
jp_embeddings = jp_embeddings.set_index('Unnamed: 0')

In [10]:
de_embeddings = de_embeddings.set_index('Unnamed: 0')

In [11]:
uk_embeddings = uk_embeddings.set_index('Unnamed: 0')

In [12]:
uk_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
913336,0.005139,0.211136,-0.020479,-0.399604,-0.209036,-0.123346,-0.233503,-0.269078,-0.054231,0.246596,...,-0.391046,-0.195277,0.234048,-0.055439,0.304113,-0.198861,-0.166215,-0.010816,-0.383944,0.291048
913337,-0.052915,-0.217416,-0.039206,0.029838,-0.130204,0.236605,-0.031174,-0.194213,0.118137,0.307406,...,0.059672,-0.354787,-0.148002,0.065507,0.398155,-0.049602,-0.641497,-0.258222,-0.186205,0.721447
913338,-0.049783,-0.081251,-0.263270,-0.405046,-0.125849,0.169087,0.164938,-0.265093,0.083483,0.090827,...,-0.193471,0.182153,-0.101107,-0.291143,0.264302,-0.069890,0.437861,0.287080,-0.336499,0.122173
913339,-0.236246,0.148378,-0.148813,0.005976,0.518185,0.309402,-0.144434,-0.180091,-0.300445,0.175444,...,-0.042600,-0.236823,0.144878,-0.010462,0.320540,-0.095306,-0.422972,0.095173,-0.198688,-0.057634
913340,-0.066016,0.196939,-0.158766,-0.128924,0.019124,0.386307,0.121630,-0.139293,-0.137734,-0.016959,...,-0.185110,-0.176911,-0.096027,0.113822,0.576839,-0.123349,-0.005054,0.181056,-0.130519,-0.035407


In [13]:
# join embeddings and products on index to get id
de_embeddings = de_embeddings.join(products_de, how='inner')

In [14]:
uk_embeddings = uk_embeddings.join(products_uk, how='inner')

In [15]:
jp_embeddings = jp_embeddings.join(products_jp, how='inner')

In [16]:
jp_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,id,title
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
518327,0.090665,-0.073974,0.022465,0.076569,0.204538,-0.012383,0.030401,0.093524,0.143870,-0.149051,...,-0.008671,-0.016234,0.173121,-0.575610,0.082543,-0.174739,0.125635,-0.173299,B0BCFTG541,ワタシガモテナイノハドウカンガエテモオマエラガワルイ022 (デジタルバンガンガンコミックス...
518328,0.023493,-0.063302,-0.017221,-0.182368,-0.011209,0.074113,0.051969,0.195828,0.087949,-0.090970,...,0.134284,-0.094964,0.117156,-0.322038,-0.166085,-0.096027,0.073333,-0.094562,B096F11HV9,"Apple MacBook Air M1 2020(13インチAir,8GB RAM,256..."
518329,0.030009,-0.014210,0.043582,-0.100854,0.035944,0.068090,0.119002,0.077852,0.051228,-0.226813,...,0.037147,-0.085642,0.445817,0.019931,-0.120491,-0.142811,0.120793,-0.062999,B0178VYR6Q,キユーピー やさしい献立 なめらかごはん 150g×6個【区分4:かまなくてよい】
518330,0.165104,-0.142523,0.065503,-0.202531,-0.058595,0.051793,0.123486,-0.059623,0.000484,-0.092261,...,0.046393,-0.026026,0.238105,0.085282,0.036479,-0.098220,0.122920,0.004804,B09VWM3SCD,Coticam outdoor ランタンシェード LED ライト ランタン キャンプ アウト...
518331,0.270576,-0.056105,0.033222,-0.011895,-0.295037,-0.094913,0.135623,0.009747,0.020907,-0.099880,...,-0.033029,-0.043809,0.484195,0.167638,-0.120333,-0.071608,0.104310,-0.061590,B07KFZPKLV,Colorsroom こたつ布団 正方形 コタツ布団 185×185cm コタツふとん 掛け...


In [17]:
de_embeddings.loc[de_embeddings['id'] == 'B08V12CT4C']

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,id,title
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
370409,-0.194557,-0.128179,-0.280744,-0.713724,-0.057707,-0.011575,-0.091817,-0.085418,-0.060973,-0.01764,...,-0.025056,0.060648,0.184876,-0.458743,-0.282159,-0.13066,-0.015066,0.193408,B08V12CT4C,ThermoPro TP49 digitales Mini Thermo-Hygromete...


In [18]:
def get_m_centroids(dim):
  # Prints possible values for m i.e. the number of centroids in the final vector in the index creation, where dim must be a multiple of m
  for n in range(1, 101):
    modulo = dim % n
    if modulo == 0:
      print(n)

In [19]:
def create_gpu_index_for_locale(embeddings, nlist, m, bits, num_samples):
    
    sample_indices = np.random.choice(embeddings.shape[0], num_samples, replace=False)
    vector_sample = embeddings[sample_indices]
    
    quantizer = faiss.IndexFlatL2(embeddings.shape[1])
    
    # initialize index
    index_cpu = faiss.IndexIVFPQ(quantizer, embeddings.shape[1], nlist, m, bits)
    index_gpu = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, index_cpu) # not needed for cpu only
    
    index_gpu.train(vector_sample)
    
    print('Index is trained {}'.format(index_gpu.is_trained))
    
    index_gpu.add(embeddings)
    
    return index_gpu

In [20]:
sessions_test_task1['prev_items'] = sessions_test_task1['prev_items'].str.strip("[']").str.replace("\n", "").str.split("' '")

In [21]:
sessions_test_task1

,prev_items,locale
0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE
1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE
2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE
3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE
4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE
...,...,...
316966,"[B077SZ2C3Y, B0B14M3VZX]",UK
316967,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",UK
316968,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",UK
316969,"[B01MCQMORK, B09JYZ325W]",UK


In [109]:
def predict_next_100_items(row, embeddings, products, k, index):

    session_embedding = embeddings.loc[row['prev_items']]
    session_embedding = session_embedding.drop(['title'], axis=1)
    session_embedding = np.mean(session_embedding, axis=0)
    session_embedding = session_embedding.values.astype('float32')
    
    D, I = index.search(np.array([session_embedding]), k)
    
    next_item = [products.loc[i]['id'] for i in I[0] if i < len(products) and i > 0] # here I[0] returns indices which are not valid indices in products
    next_item = next_item[:100]
    
    return next_item
    

# DE Locale

In [35]:
de_embeddings = de_embeddings.set_index('id')

KeyError: "None of ['id'] are in the columns"

In [37]:
index_training_de_embeddings = de_embeddings.drop(['title'], axis=1)

In [38]:
# convert dataframe to numpy array
index_training_de_embeddings = index_training_de_embeddings.values.astype('float32').copy()

In [39]:
index_training_de_embeddings.shape

(518327, 768)

In [40]:
get_m_centroids(index_training_de_embeddings.shape[1])

1
2
3
4
6
8
12
16
24
32
48
64
96


In [41]:
index_gpu_de = create_gpu_index_for_locale(index_training_de_embeddings, 100, 24, 8, 10000)

Index is trained True


In [42]:
sessions_test_task1_de = sessions_test_task1[sessions_test_task1['locale'] == "DE"]

In [110]:
sessions_test_task1_de['next_item'] = sessions_test_task1_de.apply(lambda row: predict_next_100_items(row, de_embeddings, products_de, 200, index_gpu_de), axis=1)

/scratch/slurm_tmpdir/job_22225816/ipykernel_493281/175976782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_test_task1_de['next_item'] = sessions_test_task1_de.apply(lambda row: predict_next_100_items(row, de_embeddings, products_de, 200, index_gpu_de), axis=1)


In [49]:
# sessions_test_task1_de['next_item'] = sessions_test_task1_de.apply(predict_next_100_items, args=(de_embeddings, 100, index_gpu_de), axis=1)

In [111]:
sessions_test_task1_de

,prev_items,locale,next_item
0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE,"[B08V1KXBQD, B07TV22X9M, B08V12CT4C, B099NS1XP..."
1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE,"[B00R9RZ9ZS, B00R9R5ND6, B095TL42FL, 377246997..."
2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE,"[B07G7Q5N6G, B08C9Q7QVK, B099277D7Q, B09XTMPM8..."
3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE,"[3741524131, 3955350878, 395535086X, B0041VW2D..."
4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE,"[B09J945WQR, B09J8V9RQQ, B09J8T6TTH, B09J8TWRV..."
...,...,...,...
104563,"[B014I8SSD0, B07YTX31J8, B07TVG9964, B004BEMD5...",DE,"[B00B4CVV60, B004BHZHIS, B004XISZ8E, B00SWYTH3..."
104564,"[B003M0NURK, B09VPJ4394, B07TVH9NHP]",DE,"[B08CMX8WZ6, B08CMPSCMQ, B07TVH9NHP, B09VPJ439..."
104565,"[B07B38Z3S7, B079HF6FSP, B079HF6FSP, B07B38Z3S7]",DE,"[B079MBWZS9, B079HF6FSP, B07B33GMCF, B079H8G39..."
104566,"[B00HFDYJGQ, B018G6YYDI, B00HFDYJGQ]",DE,"[B00HFDYJGQ, B00HFDYYRU, B08JH72VJG, B08JH6P9J..."


In [51]:
sessions_test_task1_de.to_csv('pred_sessions_test_task1_de.csv')

In [112]:
# Check if there is a list which does not have 100 items
list_lengths = sessions_test_task1_de['next_item'].apply(len)
indices = list_lengths[list_lengths != 100].index.tolist()

# Print the result
for i in indices:
    print(f"The length of the list at index {i} is {list_lengths[i]}")

The length of the list at index 61854 is 6
The length of the list at index 91747 is 6


# JP Locale

In [52]:
jp_embeddings = jp_embeddings.set_index('id')

In [53]:
jp_embeddings.sample()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,title
id,,,,,,,,,,,,,,,,,,,,,
B09Y86W9DD,0.062639,-0.155497,-0.015371,-0.189894,-0.063635,0.027157,0.088613,0.116732,0.11028,-0.263356,...,-0.061514,0.013169,-0.037273,0.099331,-0.291745,0.03227,0.034889,0.017256,-0.092252,ウルトラマンデッカー DX最強なりきりセット (早期購入特典 金のウルトラディメンションカー...


In [54]:
index_training_jp_embeddings = jp_embeddings.drop(['title'], axis=1)

In [55]:
# convert dataframe to numpy array
index_training_jp_embeddings = index_training_jp_embeddings.values.astype('float32').copy()

In [56]:
index_training_jp_embeddings.shape

(395009, 768)

In [57]:
get_m_centroids(index_training_jp_embeddings.shape[1])

1
2
3
4
6
8
12
16
24
32
48
64
96


In [58]:
index_gpu_jp = create_gpu_index_for_locale(index_training_jp_embeddings, 100, 24, 8, 10000)

Index is trained True


In [59]:
sessions_test_task1_jp = sessions_test_task1[sessions_test_task1['locale'] == "JP"]

In [60]:
products_jp.head()

,id,title
518327,B0BCFTG541,ワタシガモテナイノハドウカンガエテモオマエラガワルイ022 (デジタルバンガンガンコミックス...
518328,B096F11HV9,"Apple MacBook Air M1 2020(13インチAir,8GB RAM,256..."
518329,B0178VYR6Q,キユーピー やさしい献立 なめらかごはん 150g×6個【区分4:かまなくてよい】
518330,B09VWM3SCD,Coticam outdoor ランタンシェード LED ライト ランタン キャンプ アウト...
518331,B07KFZPKLV,Colorsroom こたつ布団 正方形 コタツ布団 185×185cm コタツふとん 掛け...


In [63]:
sessions_test_task1_jp['next_item'] = sessions_test_task1_jp.apply(lambda row: predict_next_100_items(row, jp_embeddings, products_jp, 100, index_gpu_jp), axis=1)

/scratch/slurm_tmpdir/job_22225816/ipykernel_493281/2871624978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_test_task1_jp['next_item'] = sessions_test_task1_jp.apply(lambda row: predict_next_100_items(row, jp_embeddings, 100, index_gpu_jp), axis=1)


In [64]:
sessions_test_task1_jp

,prev_items,locale,next_item
104568,"[B09NZHMKXW, B0823M5W31, B017GSMOJS, B09NZHMKX...",JP,"[B08NZN9WC1, B09S25SZ1Q, B06Y4F4NDS, B0BBW7QVH..."
104569,"[B08S31PW4G, B09TKN5SGL]",JP,"[B09BHP5PP1, B07F3WG12T, B09CG7D7LN, B08FB62GK..."
104570,"[B0BHWBZ4ZJ, B0BHWBZ4ZJ, B09MQCDRZ1, B07YC81BS...",JP,"[B09V7RBRWN, B000J3VKLQ, B09CD5BQ7Y, B0B7NHB4Q..."
104571,"[B07VHJZQYB, B07Y1P1B5T]",JP,"[B087CQM9BT, B089SWFFFV, B07VCLCJT3, B07K9LGGQ..."
104572,"[B09JK112HQ, B07GCTJ7JX, B0BKK2NRPJ, B07P1BH2S...",JP,"[B09FSNXWKP, B0926NWR4L, B09QXDVQ1V, B088ZW7GD..."
...,...,...,...
201030,"[B06WVR45WV, B00SMEQW6C, B00SMEQW3U]",JP,"[B005DY18Y0, B006RI3QN2, B07JZLWPQQ, B08QR8X8D..."
201031,"[B07QSHYZDR, B07QSHYZDR, B00AMUOOPK, B0044IF40...",JP,"[B0BBTP4S6N, B0B5KP97T5, B09SGBD4JF, B00FGI2QV..."
201032,"[B0146F7BBU, B06XCKFSTS, B007TFQ2QU]",JP,"[B00O9N1BEI, B0B48XHCRV, B07JG3JMCC, B005IXH6D..."
201033,"[B09SHMC6KN, B09SHMC6KN, B07PYRX4D6, B07KDH4RN...",JP,"[B07Q614Z3C, B07XGZWDZ4, B09PBD5H8X, B0B3X3FDH..."


In [65]:
sessions_test_task1_jp.to_csv('pred_sessions_test_task1_jp.csv')

# UK Locale

In [66]:
uk_embeddings = uk_embeddings.set_index('id')

In [67]:
uk_embeddings.sample()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,title
id,,,,,,,,,,,,,,,,,,,,,
B08WWPPV1P,0.216903,-0.085131,-0.03248,-0.905473,0.142746,0.315397,-0.027464,0.144623,-0.001491,0.067681,...,0.06848,0.037089,-0.120454,0.409435,0.111251,0.418192,-0.204423,0.008118,0.338016,Spiders Top Trumps Card Game


In [68]:
index_training_uk_embeddings = uk_embeddings.drop(['title'], axis=1)

In [69]:
# convert dataframe to numpy array
index_training_uk_embeddings = index_training_uk_embeddings.values.astype('float32').copy()

In [70]:
index_training_uk_embeddings.shape

(500180, 768)

In [71]:
get_m_centroids(index_training_uk_embeddings.shape[1])

1
2
3
4
6
8
12
16
24
32
48
64
96


In [72]:
index_gpu_uk = create_gpu_index_for_locale(index_training_uk_embeddings, 100, 24, 8, 10000)

Index is trained True


In [73]:
sessions_test_task1_uk = sessions_test_task1[sessions_test_task1['locale'] == "UK"]

In [74]:
products_uk.head()

,id,title
913336,B087LZNPHS,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid..."
913337,B08THFN1KX,Hippowarehouse Personalised Photo Printed Mous...
913338,0804185328,"500 Easy Recipes for Every Machine, Both Stove..."
913339,B09VBKDBW6,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S..."
913340,B096ZW8B49,Lucosobie Steering Wheel Lock - Car Anti-Theft...


In [75]:
sessions_test_task1_uk['next_item'] = sessions_test_task1_uk.apply(lambda row: predict_next_100_items(row, uk_embeddings, products_uk, 100, index_gpu_uk), axis=1)

/scratch/slurm_tmpdir/job_22225816/ipykernel_493281/2720364193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_test_task1_uk['next_item'] = sessions_test_task1_uk.apply(lambda row: predict_next_100_items(row, uk_embeddings, 100, index_gpu_uk), axis=1)


In [76]:
sessions_test_task1_uk

,prev_items,locale,next_item
201035,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",UK,"[B091TLGK2T, B0BFX4PPYM, B08WPCZ8KR, B09J95XLG..."
201036,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",UK,"[B0791BS4NL, B09SH64F6S, B07S7CZZ7L, B09WZ4FJ3..."
201037,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5...",UK,"[B0B7RW8HYV, B07TS1NXSH, B07MMWKWH1, B08HFJJTJ..."
201038,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]",UK,"[B0B717ZVD5, B006L4DLW8, B091F4QN7K, B09PF9CPN..."
201039,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",UK,"[3742322508, B095XWVYMN, B001QCELXS, B00629V1D..."
...,...,...,...
316966,"[B077SZ2C3Y, B0B14M3VZX]",UK,"[B08TVMDJJ7, B088C252ZG, B017XZKLCG, B081RJWHB..."
316967,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",UK,"[B09HH12F4H, B08GCSTGCN, B09GBLVL8S, B07H4RXYN..."
316968,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",UK,"[B087PFHYSB, B089WCZ5YN, B0B7NTZ4N4, B0B7JBZ9K..."
316969,"[B01MCQMORK, B09JYZ325W]",UK,"[B09SDJTL2Z, B093GH3HK6, B07GLMVWTB, B09FF8VS6..."


In [77]:
sessions_test_task1_uk.to_csv('pred_sessions_test_task1_uk.csv')

In [100]:
sessions_test_task1_uk

,prev_items,locale,next_item
201035,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",UK,"[B091TLGK2T, B0BFX4PPYM, B08WPCZ8KR, B09J95XLG..."
201036,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",UK,"[B0791BS4NL, B09SH64F6S, B07S7CZZ7L, B09WZ4FJ3..."
201037,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5...",UK,"[B0B7RW8HYV, B07TS1NXSH, B07MMWKWH1, B08HFJJTJ..."
201038,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]",UK,"[B0B717ZVD5, B006L4DLW8, B091F4QN7K, B09PF9CPN..."
201039,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",UK,"[3742322508, B095XWVYMN, B001QCELXS, B00629V1D..."
...,...,...,...
316966,"[B077SZ2C3Y, B0B14M3VZX]",UK,"[B08TVMDJJ7, B088C252ZG, B017XZKLCG, B081RJWHB..."
316967,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",UK,"[B09HH12F4H, B08GCSTGCN, B09GBLVL8S, B07H4RXYN..."
316968,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",UK,"[B087PFHYSB, B089WCZ5YN, B0B7NTZ4N4, B0B7JBZ9K..."
316969,"[B01MCQMORK, B09JYZ325W]",UK,"[B09SDJTL2Z, B093GH3HK6, B07GLMVWTB, B09FF8VS6..."


In [90]:
# Check if there is a list which does not have 100 items
list_lengths = sessions_test_task1_uk['next_item'].apply(len)
indices = list_lengths[list_lengths != 100].index.tolist()

# Print the result
for i in indices:
    print(f"The length of the list at index {i} is {list_lengths[i]}")

The length of the list at index 203493 is 46
The length of the list at index 207183 is 46
The length of the list at index 209591 is 46
The length of the list at index 210756 is 46
The length of the list at index 211906 is 46
The length of the list at index 216342 is 46
The length of the list at index 216643 is 46
The length of the list at index 218460 is 81
The length of the list at index 227131 is 50
The length of the list at index 239352 is 46
The length of the list at index 239645 is 81
The length of the list at index 246330 is 81
The length of the list at index 246434 is 50
The length of the list at index 251718 is 46
The length of the list at index 255477 is 46
The length of the list at index 280548 is 81
The length of the list at index 281766 is 46
The length of the list at index 285119 is 46
The length of the list at index 291193 is 46
The length of the list at index 294873 is 50
The length of the list at index 311125 is 46
The length of the list at index 314952 is 81


# Combine predictions